In [27]:
import selenium as sel # automation tool
from selenium.webdriver.common.by import By
from selenium import webdriver # automation tool
import time # doing timeouts
from bs4 import BeautifulSoup # parsing tool
import warnings # warning so my head don't explode
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd # data analysis tool
import numpy as np # numerical analysis tool
import requests # web requests tool
import re # regular expression tool

In [30]:
restaurant_list = ['Green China Grill', 'Sizzling Pot King'] # list of restaurant names
df_comments = pd.DataFrame(columns=['restaurant', 'comment']) # create dataframe to store comments
driver = webdriver.Safari() # create driver

for resturant in restaurant_list:
    print(resturant) 
    try:
        driver.get('https://www.yelp.com/') # open yelp website
        time.sleep(2) # wait 2 seconds to make sure page is loaded
        elem = driver.find_element(By.XPATH, "//input[contains(@id, 'search_description')]") # find search bar
        elem.send_keys(resturant) # search for restaurant
        elem = driver.find_element(By.XPATH, "//button[contains(@class, 'find-near-button')]") # find search button
        elem.click() # click search button
        time.sleep(2) # wait 2 seconds to make sure page is loaded
        original_window = driver.current_window_handle # save original window
        elem = driver.find_elements(By.XPATH, '//div[contains(@class, "businessName")]')[1] # find restaurant with correct name
        elem.click() # click restaurant
        time.sleep(2) # wait 2 seconds to make sure page is loaded

        # switch to new window
        for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break

        while True: 
            url = driver.current_url # get url 
            req = requests.get(url) # get html
            soup = BeautifulSoup(req.content, 'html.parser') # parse html
            comments = soup.find_all('p', class_ = re.compile('comment.*')) # find all comments

            # store comments in dataframe
            for comment in comments:
                df_comments = df_comments.append({'restaurant': resturant, 'comment': comment.text}, ignore_index=True)
                
            elem = driver.find_elements(By.XPATH, "//a[contains(@class, 'next-link navigation-button')]")[0] # find next page button
            elem.click() 
    except: 
        continue # no more pages

Green China Grill
Sizzling Pot King


In [34]:
df_comments.to_csv('comments.csv', index=False) # save dataframe to csv